In [ ]:
import torch
import torch.autograd as autograd
from torch import Tensor
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
import scipy.io

import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float)

torch.manual_seed(1234)
np.random.seed(1234)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
data = pd.read_csv('bio.csv', sep=',')
data = data.replace(np.nan, 0.)

In [ ]:
t = data['t'].to_numpy().reshape(-1,1)
cB = data['cB'].to_numpy().reshape(-1,1)
cTG = data['cTG'].to_numpy().reshape(-1,1)
cDG = data['cDG'].to_numpy().reshape(-1,1)
cMG = data['cMG'].to_numpy().reshape(-1,1)
cG = data['cG'].to_numpy().reshape(-1,1)
c = np.concatenate((cB, cTG, cDG, cMG, cG), axis=1)
c

In [ ]:
total_points = 4

t_train = np.linspace(t[0], t[-1], total_points)

idx = []
for ti in t:
    idx.append(np.where(t_train.reshape(-1,1) == ti)[0][0])
idx = np.array(idx)
idx

In [ ]:
cTG_train = cTG
cDG_train = cDG
cMG_train = cMG
cTG_train

In [ ]:
t_train = torch.from_numpy(t_train).float().to(device)
cTG_train = torch.from_numpy(cTG_train).float().to(device)
cDG_train = torch.from_numpy(cDG_train).float().to(device)
cMG_train = torch.from_numpy(cMG_train).float().to(device)
cTG_train

In [ ]:
k1 = 1.
k2 = 1.
k3 = 1.
k4 = 1.
k5 = 1.
k6 = 1.

layers = np.array([1,10,10,10,1])

f_hat = torch.zeros(t_train.shape[0],1).to(device)

alpha = 0.5

In [ ]:
class DNN(nn.Module):
     
    def __init__(self, layers):
        super().__init__()
        
        self.activation = nn.Tanh()

        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])

        for i in range(len(layers)-1):
            
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            nn.init.zeros_(self.linears[i].bias.data)
            
    def forward(self, x):
              
        if torch.is_tensor(x) != True:
            x = torch.from_numpy(x)
        
        a = x.float()
        
        for i in range(len(layers)-2):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a1 = self.linears[-1](a)
        a2 = self.linears[-1](a)
        a3 = self.linears[-1](a)
        a4 = self.linears[-1](a)
        a5 = self.linears[-1](a)
        
        return a1, a2, a3, a4, a5

DNN(layers)

In [ ]:
class FCN():
    
    def __init__(self, layers):
        self.dnn = DNN(layers).to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')

        self.iter = 0

        self.k1 = torch.tensor([k1], requires_grad=True).float().to(device)
        self.k2 = torch.tensor([k2], requires_grad=True).float().to(device)
        self.k3 = torch.tensor([k3], requires_grad=True).float().to(device)
        self.k4 = torch.tensor([k4], requires_grad=True).float().to(device)
        self.k5 = torch.tensor([k5], requires_grad=True).float().to(device)
        self.k6 = torch.tensor([k6], requires_grad=True).float().to(device)

        self.k1 = nn.Parameter(self.k1)
        self.k2 = nn.Parameter(self.k2)
        self.k3 = nn.Parameter(self.k3)
        self.k4 = nn.Parameter(self.k4)
        self.k5 = nn.Parameter(self.k5)
        self.k6 = nn.Parameter(self.k6)

        self.dnn = DNN(layers).to(device)
        
        self.dnn.register_parameter('k1', self.k1)
        self.dnn.register_parameter('k2', self.k2)
        self.dnn.register_parameter('k3', self.k3)
        self.dnn.register_parameter('k4', self.k4)
        self.dnn.register_parameter('k5', self.k5)
        self.dnn.register_parameter('k6', self.k6)
        
    def loss_cB(self, x):
        
        g = x.clone()
        g.requires_grad = True
        
        cB, cTG, cDG, cMG, cG = self.dnn(g)
        
        grad_cB = autograd.grad(cB, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cB_ode = self.loss_function(grad_cB - self.k1*cTG + self.k2*cDG*cB \
                                                 - self.k3*cDG + self.k4*cMG*cB \
                                                 - self.k5*cMG + self.k6*cG*cB, f_hat)
        
        return loss_cB_ode
    
    def loss_cTG(self, x, y):
        
        g = x.clone()
        g.requires_grad = True
        
        cB, cTG, cDG, _, _ = self.dnn(g)
        
        grad_cTG = autograd.grad(cTG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cTG_ode = self.loss_function(grad_cTG + self.k1*cTG - self.k2*cDG*cB, f_hat)
        
        loss_cTG_data = self.loss_function(cTG, y)
        
        return alpha*loss_cTG_ode + (1-alpha)*loss_cTG_data
    
    def loss_cDG(self, x, y):
        
        g = x.clone()
        g.requires_grad = True
        
        cB, cTG, cDG, cMG, _ = self.dnn(g)
        
        grad_cDG = autograd.grad(cDG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cDG_ode = self.loss_function(grad_cDG - self.k1*cTG + self.k2*cDG*cB \
                                                   + self.k3*cDG - self.k4*cMG*cB, f_hat)
        
        loss_cDG_data = self.loss_function(cDG, y)
        
        return alpha*loss_cDG_ode + (1-alpha)*loss_cDG_data
    
    def loss_cMG(self, x, y):
        
        g = x.clone()
        g.requires_grad = True
        
        cB, _, cDG, cMG, cG = self.dnn(g)
        
        grad_cMG = autograd.grad(cMG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cMG_ode = self.loss_function(grad_cMG - self.k3*cDG + self.k4*cMG*cB \
                                                   + self.k5*cMG - self.k6*cG*cB, f_hat)
        
        loss_cMG_data = self.loss_function(cMG, y)
        
        return alpha*loss_cMG_ode + (1-alpha)*loss_cMG_data
    
    def loss_cG(self, x):
        
        g = x.clone()
        g.requires_grad = True
        
        cB, _, _, cMG, cG = self.dnn(g)
        
        grad_cG = autograd.grad(cG, g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        loss_cG_ode = self.loss_function(grad_cG - self.k5*cMG + self.k6*cG*cB, f_hat)
                
        return loss_cG_ode
    
    def closure(self):
        
        optimizer.zero_grad()
        
        loss_u1 = self.loss_cB(t_train)
        loss_u2 = self.loss_cTG(t_train, cTG_train)
        loss_u3 = self.loss_cDG(t_train, cDG_train)
        loss_u4 = self.loss_cMG(t_train, cMG_train)
        loss_u5 = self.loss_cG(t_train)
        
        loss = loss_u1 + loss_u2 + loss_u3 + loss_u4 + loss_u5
        
        loss.backward()
                
        self.iter += 1
        
        return loss

In [ ]:
PINN = FCN(layers)

PINN.dnn(t_train)

In [ ]:
params = list(PINN.dnn.parameters())

optimizer = torch.optim.Adam(params, lr=1)

while PINN.iter < 100:
    optimizer.step(PINN.closure)
    for p in PINN.dnn.parameters():
        p.data.clamp_(min=0.)
        
print(PINN.dnn(t_train))

In [ ]:
def EDO(y, prm):
    
    k1 = prm[0]
    k2 = prm[1]
    k3 = prm[2]
    k4 = prm[3]
    k5 = prm[4]
    k6 = prm[5]
    
    f = np.zeros(5)
    
    f[1] = - k1*y[1] + k2*y[2]*y[0]
    f[2] = + k1*y[1] - k2*y[2]*y[0] - k3*y[2] + k4*y[3]*y[0]
    f[3] = + k3*y[2] - k4*y[3]*y[0] - k5*y[3] + k6*y[4]*y[0]
    f[4] = + k5*y[3] - k6*y[4]*y[0]
    f[0] = + k1*y[1] - k2*y[2]*y[0] + k3*y[2] - k4*y[3]*y[0] + k5*y[3] - k6*y[4]*y[0]
    
    return f

def euler_explicite(y0, dt, tf, prm):
    
    mat_y = np.array([y0])
    
    t = np.array([0])
    while t[-1] < tf:
        y = y0 + dt * EDO(y0, prm)
        
        mat_y = np.append(mat_y, [y], axis=0)
        t = np.append(t, t[-1]+dt)
        
        y0 = np.copy(y)
    
    return t, mat_y

prm = []        
for i in range(6):
    prm.append(params[:6][i][0].cpu().detach().numpy())
    
t_euler, y_euler = euler_explicite(np.array([0,0.540121748,0.057018273,0,0]), 0.001, 6, prm)

In [ ]:
plt.plot(t_euler, y_euler[:,0])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[0].detach().numpy())
plt.plot(t, cB, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,1])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[1].detach().numpy())
plt.plot(t, cTG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,2])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[2].detach().numpy())
plt.plot(t, cDG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,3])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[3].detach().numpy())
plt.plot(t, cMG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,4])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train)[4].detach().numpy())
plt.plot(t, cG, 'o')
plt.show()

In [ ]:
prm